In [14]:
# importing libraries

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# setting randoms state
tf.random.set_seed(42)

In [46]:
# Time series to LSTM data
def timeseries_to_supervised(data, lag=1):
    df = pd.DataFrame(data)
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = pd.concat(columns, axis=1)
    df.fillna(0, inplace=True)
    return df.to_numpy()

# # Differenced series
# def difference(dataset, interval=1):
#     diff = list()
#     for i in range(interval, len(dataset)):
#         value = dataset[i] - dataset[i - interval]
#         diff.append(value)
#     return pd.Series(diff)


# # Undo differenced series
# def inverse_difference(history, yhat, interval=1):
#  return yhat + history[-interval]

In [50]:
# reading data
df_train = pd.read_csv('data_v1.1.csv')

# aggregatin data
df_train["datetime"] = pd.to_datetime(df_train["datetime"], format="%d/%m/%Y %H:%M")
df_train = df_train[["datetime", "ppl_out", "ppl_in"]].groupby(by="datetime").sum()
df_train["Net flow"] = df_train["ppl_in"] - df_train["ppl_out"]

# scaling data
scaler = MinMaxScaler(feature_range=(-1, 1))
scaled = scaler.fit_transform(df_train[["Net flow"]])

# train-test split
train_size = int(len(scaled) * 0.8)
test_size = len(scaled) - train_size
train, test = scaled[0:train_size,:], scaled[train_size:,:]

# LSTM dimensions
lag = 7
train_lagged = timeseries_to_supervised(train, lag)
X, y = train_lagged[:,0:-1], train_lagged[:,-1]
X = X.reshape(X.shape[0], 1, X.shape[1])

In [52]:
# LSTM model

neurons = 3
batch_size = 1
epochs = 500

model = Sequential()
model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=1, shuffle=False)

Epoch 1/500
3605/3605 [==============================] - 4s 866us/step - loss: 0.2712
Epoch 2/500
3605/3605 [==============================] - 3s 876us/step - loss: 0.2611
Epoch 3/500
3605/3605 [==============================] - 3s 925us/step - loss: 0.2604
Epoch 4/500
3605/3605 [==============================] - 3s 890us/step - loss: 0.2599
Epoch 5/500
3605/3605 [==============================] - 4s 1ms/step - loss: 0.2596
Epoch 6/500
3605/3605 [==============================] - 3s 922us/step - loss: 0.2594
Epoch 7/500
3605/3605 [==============================] - 3s 896us/step - loss: 0.2592
Epoch 8/500
3605/3605 [==============================] - 3s 883us/step - loss: 0.2589
Epoch 9/500
3605/3605 [==============================] - 3s 875us/step - loss: 0.2587
Epoch 10/500
3605/3605 [==============================] - 4s 1ms/step - loss: 0.2584
Epoch 11/500
3605/3605 [==============================] - 6s 2ms/step - loss: 0.2582
Epoch 12/500
3605/3605 [==============================] - 

KeyboardInterrupt: 